## 주차 탐지 프로젝트 - *YOLOv8을 통한 추가 세그멘테이션 적용*

### 1. **프로젝트 배경 및 목표**
- 본 프로젝트는 주차 공간 탐지를 목적으로 하며, 이를 위해 AI허브에서 제공하는 주차 공간 관련 데이터셋을 활용함. 
- 해당 데이터셋은 기본적으로 주차 가능 영역(Parking)과 주차 불가능 영역(Non-Parking)에 대한 세그멘테이션 정보만을 포함하고 있음.

### 2. **문제점**
- 기존 데이터셋은 주차 가능 여부만을 구분하는 데 초점이 맞춰져 있어, **사람(Person)** 이나 **차량(Vehicle)** 과 같은 추가 객체를 탐지하는 것에 한계를 가짐. 
- 하지만 해당 객체들의 탐지는 주차 공간의 정확한 식별과 시스템의 실용성을 향상시키는 데 중요한 요소로 작용할 수 있음. 특히 사람(Person), 오토바이(Motorcycle), 자전거(Bicycle) 등이 걸쳐져 있는 공간을 주차 가능 공간으로 오인하는 상황이 발생할 수도 있음.

### 3. **해결 방안**
- 이 문제를 해결하고자, **YOLOv8** 모델을 사용하여 기존 데이터셋에 **사람(Person)** 과 **차량(Vehicle)** 객체를 추가로 세그멘테이션할 수 있도록 확장함.

### 4. **YOLOv8 모델 선택 이유**
- YOLOv8은 실시간 탐지에 강점을 지니고 있어, 복잡한 배경 속 다양한 객체를 빠르고 정확하게 탐지할 수 있음. 이를 통해 주차 공간과 관련된 다양한 객체들도 효과적인 식별을 기대할 수 있음.

### 5. **기본 작업 순서**

a. **`cocotrans_yolo.py`로 데이터셋 coco 포맷 추출**
   - COCO 포맷의 데이터셋을 생성

b. **`get_yolo_segmentation.py`로 person, vehicle txt 좌표 파일 추출**
   - Person과 Vehicle 클래스의 YOLO 좌표 파일을 추출
   - 이후 runs/ 폴더 생성 확인

c. **`yolo_seg_in_coco.py`로 YOLO 좌표를 COCO JSON 구조로 변경**
   - 이전 단계에서 추출된 YOLO 좌표 파일 COCO JSON 구조로 변환

d. **`one_coco_yolo.py`로 카테고리 4개 고정**
   - 카테고리를 4개로 고정한 후, 결과 파일의 이름을 merge_yolo_coco.json으로 변경

e. **`check.py` 파일 실행** 
   - 카테고리 4개가 올바르게 설정되었는지 확인(선택사항)

f. **`model.train` 실시**
   - 학습이 완료된 가중치 파일의 이름 best_yolo_model.pth로 변경 (기본 데이터셋 가중치, 학습 결과 비교 실시)
   - 테스트 사진은 night, 학습 결과는 result_night 파일로 저장


------------------------------------------------------------------------------------------------------------------------------------------------------

### 1. **`cocotrans_yolo.py`로 데이터셋 coco 포맷 추출**

In [ ]:
import os
import json
import cv2
import numpy as np

In [ ]:
def calculate_area(polygon):
    x = np.array(polygon[::2])
    y = np.array(polygon[1::2])
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def calculate_bbox(polygon):
    x = polygon[::2]
    y = polygon[1::2]
    return [min(x), min(y), max(x) - min(x), max(y) - min(y)]

**def calculate_area(polygon)**
- x1,y1,x2,y2... 구조로 정리된 폴리곤 좌표 리스트를 계산함. 
- 예시로 아래와 같은 폴리곤 좌표라면 x : [974.5, 1125.85, 1274.73, 1021.27], y : [725.65, 734.6, 787.7, 776.35] 로 추출할 수 있음. 
- 이후 x,y 좌표를 순차적으로 곱한 내적의 차이를 이용하는 'Shoelace' 공식을 통해 최종 결과 값을 반환함. 

**def calculate_bbox(polygon)**
- 폴리곤 좌표로 생성되는 최소 상자모양 경계를 계산함.
- x : [974.5, 1125.85, 1274.73, 1021.27], y : [725.65, 734.6, 787.7, 776.35] 라면, 최종적으로 [974.5, 725.65, 1274.73 - 974.5, 787.7 - 725.65] 좌표를 반환함.

**<폴리곤 좌표 예시>**
```python
polygon = [
    [974.5, 725.65],
    [1125.85, 734.6],
    [1274.73, 787.7],
    [1021.27, 776.35]
]

In [ ]:
def convert_json_to_coco(input_json_path, image_dir, output_dir):
    
    # 1. JSON 파일 열기 및 데이터 로드
    with open(input_json_path, "r") as f:
        data = json.load(f)


    img_filename = os.path.basename(input_json_path).replace(".json", ".jpg")
    width = 1920
    height = 1080

    # 2. COCO 포맷 기본 구조 생성
    coco = {
        "images": [{
            "id": 0,
            "file_name": img_filename,
            "width": width,
            "height": height
        }],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "Parking Space"},
            {"id": 2, "name": "Driveable Space"}
        ]
    }

    annotation_id = 0
    category_id_map = {cat['name']: cat['id'] for cat in coco['categories']}

    # 3. 세그멘테이션 정보 추출 및 변환
    for segmentation in data.get("segmentation", []):
        category_name = segmentation["name"]
        if category_name not in category_id_map:
            continue  # 'Parking Space'나 'Driveable Space'가 아니면 무시

        category_id = category_id_map[category_name]

        # 폴리곤 좌표를 1차원 리스트로 변환!! 
        polygon = [coord for point in segmentation["polygon"] for coord in point]

        annotation = {
            "id": annotation_id,
            "image_id": 0,
            "category_id": category_id,
            "segmentation": [polygon],
            "area": calculate_area(polygon),
            "bbox": calculate_bbox(polygon),
            "iscrowd": 0
        }
        coco["annotations"].append(annotation)
        annotation_id += 1

    # 4. 새로운 JSON 파일로 저장
    output_filename = os.path.basename(input_json_path).replace(".json", "_coco.json")
    output_path = os.path.join(output_dir, output_filename)

    with open(output_path, "w") as f:
        json.dump(coco, f, indent=4)

**def convert_json_to_coco(input_json_path, image_dir, output_dir)**
- JSON 형식의 데이터를 COCO 포맷으로 변경함으로서 욜로와 같은 딥러닝 모델의 호환성을 높이고자 함. 특히 최종 모델 학습에서 우리 팀은 Torch 기반 MASK R-cnn을 사용하기 때문에 COCO 포맷으로 변환함.

**#1. JSON 파일 열기 및 데이터 로드**
- 'input_json_path' 를 통해 이미지, 세그멘테이션 정보가 포함된 json파일을 불러옴. 

**#2. COCO 포맷 기본 구조 생성**
- COCO 포맷의 기본 구조를 설정해 줌. 현재 본 프로젝트에서 사용하려고 하는 데이터는 'images', 'annotations', 'categories'가 포함되어 있으며 'categories' id 3,4번에 각각 "Person", "Vehicles"를 추가할 예정임.

**#3. 세그멘테이션 정보 추출 및 변환** 
- JSON 데이터의 세그멘테이션 정보를 읽어와서 COCO 포맷의 annotations에 맞게 변환함. 특히 세그멘테이션 정보에서 폴리곤 좌표를 1차원 리스트로 변환함. 
- 앞서 언급되었 듯, COCO 데이터 포맷의 좌표는 다각형 좌표를 [x1, y1, x2, y2, x3, y3..] 형식으로 저장하기 때문에 1차원 리스트로 변환하는게 맞음.

**#4. 새로운 JSON 파일로 저장**
- 해당 함수의 결과값을 이후 '_coco.json' 형태로 바꿔서 output_path로 추출함.

In [ ]:
# 사용 예시
input_json_dir = "/home/elicer/KSH/segmentjson"  # JSON 파일들이 있는 디렉토리
image_dir = "/home/elicer/KSH/segmentimage"  # 이미지 파일들이 있는 디렉토리
output_dir = "/home/elicer/KSH/coco_files"  # COCO 포맷으로 저장할 디렉토리

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_json_dir):
    if filename.endswith(".json"):
        convert_json_to_coco(os.path.join(input_json_dir, filename), image_dir, output_dir)


**'input_json_dir', 'image_dir', 'output_dir'**
- 결론적으로 `cocotrans_yolo.py` 파일을 사용하기 위해서는 세가지 변수가 필요함.
- input_json_dir 에는 세그멘테이션 좌표가 포함된 JSON 파일이, image_dir에는 앞선 JSON과 한쌍인 이미지 파일이 필요함. 경로 관련 예시는 위의 코드를 참고하면 됨.
- 일련의 과정을 거쳐 output_dir에 할당된 경로로 _coco.json으로 변환된 파일 확인 가능함.

------------------------------------------------------------------------------------------------------------------------------------------------------

### 2. **`get_yolo_segmentation.py`로 person, vehicle txt 좌표 파일 추출**

In [ ]:
from ultralytics import YOLO
import os

model = YOLO("yolov8n-seg.pt") 
#n = model.names
#print(n)

folder = r'/home/elicer/merged_data/merged_segmentimage'    # 이미지 파일의 폴더 경로
image_extensions = {'.jpg'}  # 허용된 이미지 --> 나중에 추후 이미지 외의 데이터도 처리 가능(확장성 생각)

for filename in os.listdir(folder):
    # 파일 확장자 형식만 처리
    if os.path.splitext(filename)[-1].lower() in image_extensions:
        filepath = os.path.join(folder, filename)
        result = model.predict(filepath, device='cuda', save=True, save_txt=True, conf=0.5)


앞서 언급했듯이 현재 가지고 있는 데이터셋으로는 사람, 자전거, 트럭 등이 포함된 복잡한 환경에서 이미지 내 객체 탐지의 정확성이 떨어질 수 있음. 따라서 해결책으로 JSON 파일 내 'Parking Space', 'Driveable Space' 외에 'Person', 'Vehicle'을 추가하는 방안을 택함.

<br>

**model = YOLO("yolov8n-seg.pt")**
- 세그멘테이션 기능이 포함된 YOLOv8 모델을 사용함. 해당 모델은 이미 'Person', 'Vehicle' 클래스에 대해 학습된 가중치를 포함하고 있기 때문에 일정부분 정확성을 가졌으며, 향후 최종 학습 모델로 학습 시 보유한 데이터셋으로 더 나은 가중치를 얻을 수 있을 것으로 예상됨.

**result = model.predict(filepath, device='cuda', save=True, save_txt=True, conf=0.5)**
- 본 프로젝트 작업 환경은 nvidia a100으로 연산작업의 횽율성을 고려해 gpu를 사용하였음. 그 외에 'save=True', 'save_txt=True'로 각각 시각적, 이미지 데이터를 저장함. 
- 'conf=0.5'는 예측에 대한 최소 신뢰도(confidence)로 해당 모델에서는 50%으로 설정함. 만약 모델이 탐지한 객체가 신뢰도(50%)보다 낮을 경우 해당 객체는 예측에서 제외하는 방식으로 진행함.
- `get_yolo_segmentation.py`를 통해 얻을 수 있는 결과는 시각적 데이터와 텍스트 데이터 두 가지임. 현 작업에서 필요한 것은  'Person', 'Vehicle' 등의 폴리곤 좌표가 포함된 세그멘테이션 결과이며, 이는 .txt 파일로 저장됨. 해당 텍스트 파일에는 각 객체의 클래스, 위치, 신뢰도 등의 정보를 담고 있어 향후 데이터 분석 혹은 모델 개선 등 확장성 측면에서 용이함.

------------------------------------------------------------------------------------------------------------------------------------------------------

### 3.  **`yolo_seg_in_coco.py`로 YOLO 좌표를 COCO JSON 구조로 변경**

In [ ]:
import json
import os

# YOLO-seg.py 에서 얻은 txt폴더 경로 이용!
yolo_segments_folder ='/home/elicer/runs/segment/predict/labels'
# parking space, driveable space만 있는 해당 이미지에 맞는 coco.json 파일 가져오기
coco_json_folder = '/home/elicer/merged_data/coco_files'
# 변환된 COCO JSON 파일을 저장할 경로
output_folder = '/home/elicer/runs/segment/predict/cocojson'

**'yolo_segments_folder', 'coco_json_folder', 'output_folder'**
- `get_yolo_segmentation.py`에서 추출한 이미지 좌표 텍스트 파일을 COCO 포맷 JSON 파일로 변경하는 작업이 필요함.

<br>

- 따라서 필요한 변수는 총 2가지로 'yolo_segments_folder'은 이전 작업에서 생성된 .txt 형식의 파일들이 저장된 폴더이며, 'coco_json_folder'은 `cocotrans_yolo.py`에서 추출한 COCO 포맷 JSON 파일이 저장된 폴더로 'Parking Space', 'Driveable Space' 카테고리 정보만 포함하고 있음
- output_folder로는 최종적으로 추출된 COCO.JSON 파일을 저장할 위치를 설정해 줌.

In [ ]:
def convert_yolo_to_coco(segment, image_width, image_height):
    coco_segment = []
    for i in range(1, len(segment), 2):  
        x = segment[i] * image_width
        y = segment[i + 1] * image_height
        coco_segment.extend([x, y])
    return coco_segment

**def convert_yolo_to_coco(segment, image_width, image_height)**
- 함수에 포함된 각 변수를 살펴보면 'segment'는 YOLO에서 생성된 세그멘테이션 좌표로 0~1 사이의 값을 가지며 이미지 크기에 대한 상대적인 좌표를 나타냄. 'image_width', 'image_height'는 각각 이미지 너비, 높이를 의미함.
- 계속 언급 되듯이 YOLO 좌표는 상대적인 좌표로 절대적 좌표로의 변환이 필요함. 이를 위해 for문을 이용해 (image_width, image_height)와 곱해 절대좌표를 추출함.

<br>

- **YOLO 포맷 예시**: 아래 예시에서 맨 앞의 2는 클래스 ID(Driveable Space)이고, 그 뒤에 나오는 x, y 좌표가 0~1 사이의 상대값으로 나타난 것을 확인할 수 있습니다.
- **COCO 포맷 예시**: COCO 포맷에서는 YOLO 포맷의 상대적 좌표에서 변환된 절대 좌표로 나타남. 예를 들어, YOLO 포맷의 '0.604688'이라는 값은 이미지 너비와 곱해져 '1161.00096'과 같은 절대 좌표로 변환되었음.



<br>

- [참고] 클래스 ID 목록:
    - ID 1: Parking Space
    - ID 2: Driveable Space
    - ID 3: Person
    - ID 4: Vehicle

<br>

**<'image_10.txt' YOLO 포맷 예시>**
```python
2 0.604688 0.527778 0.604688 0.536111 ...

**<'image_10_coco_updated.json ' COCO 포맷 예시>**
```python
{
    "id": 5,
    "image_id": 0,
    "category_id": 4,
    "segmentation": [
        [
            1161.00096,
            570.00024,
            1161.00096,
            578.99988,
            ...
        ]
    ],
    ...
}

In [ ]:
# output folder 생성
os.makedirs(output_folder, exist_ok=True)

#1. COCO JSON 파일과 YOLO txt 파일 매칭 및 JSON 데이터 로드
for coco_file in os.listdir(coco_json_folder):
    if coco_file.endswith(".json"):
        coco_json_file = os.path.join(coco_json_folder, coco_file)
        yolo_segments_name = os.path.basename(coco_json_file).replace("_coco.json", ".txt")
        yolo_segments_file = os.path.join(yolo_segments_folder, yolo_segments_name)
        
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)

        #2. YOLO .txt 파일 읽어오기 및 좌표 변환   
        if os.path.exists(yolo_segments_file): 
            yolo_segments = []
            with open(yolo_segments_file, 'r') as f:
                for line in f:
                    segment = list(map(float, line.strip().split()))                     
                    yolo_segments.append(segment)

            #3. 새 어노테이션 생성 및 추가
            existing_ids = [annotation['id'] for annotation in coco_data['annotations']]
            new_id = max(existing_ids) + 1 if existing_ids else 0

            # 이미지 크기 (기존 JSON에서 가져옴)
            image_info = coco_data['images'][0]
            image_width = image_info['width']                  
            image_height = image_info['height']

            # YOLO 좌표를 COCO 형식으로 변환하고 annotations에 추가
            for segment in yolo_segments:
                category_id = 3 if int(segment[0]) == 0 else 4 if int(segment[0]) == 2 else int(segment[0])
                coco_segment = convert_yolo_to_coco(segment, image_width, image_height)

                # 새 annotation 생성
                annotation = {
                    "id": new_id,
                    "image_id": image_info['id'],
                    "category_id": category_id,
                    "segmentation": [coco_segment],
                    "area": 0,  # 면적 계산은 필요 시 추가
                    "bbox": [],  # Bounding box 계산은 필요 시 추가
                    "iscrowd": 0
                }

                # bbox와 area 계산
                xs = coco_segment[::2]
                ys = coco_segment[1::2]
                x_min, x_max = min(xs), max(xs)
                y_min, y_max = min(ys), max(ys)
                bbox = [x_min, y_min, x_max - x_min, y_max - y_min]
                area = (x_max - x_min) * (y_max - y_min)
                annotation['bbox'] = bbox
                annotation['area'] = area

                # COCO 데이터에 새로운 annotation 추가
                coco_data['annotations'].append(annotation)

                # ID 증가
                new_id += 1

            #4. 새로운 카테고리 추가
            new_categories = [
                {"id": 3, "name": "Person"},
                {"id": 4, "name": "Vehicle"}
            ]

            for category in new_categories:
                if category not in coco_data['categories']:
                    coco_data['categories'].append(category)
    
        #5. 수정된 JSON 파일 저장
        output_file = os.path.join(output_folder, f"{os.path.splitext(yolo_segments_name)[0]}_coco_updated.json")
        with open(output_file, 'w') as f:
            json.dump(coco_data, f, indent=4)      


**#1. COCO JSON 파일과 YOLO .txt 파일 매칭 및 JSON 데이터 로드**
- 먼저 기존(paking space, driveable space) COCO JSON 파일을 순회하며 YOLO .txt 파일과의 매칭이 필요함. 예를들어 'image_10_coco.json'이라는 파일이 있으면 'image_10.txt'라는 YOLO 파일을 찾아서 연결하도록 함.  

**#2. YOLO txt 파일 읽기 및 좌표 변환**
- #1의 순회를 통해 텍스트 파일이 존재하는 경우 해당 파일을 읽어 yolo_segment=[] 리스트에 좌표를 저장함. 

**#3. 새 어노테이션 생성 및 추가** 
- 이 부분에서는 기존 JSON 파일에서 ID 확인 후 'Person', 'Vehicle' ID를 추가하는 작업을 실시함. 이번 모델 학습에서 바운딩박스를 따로 이용하지는 않지만 추후 확장성을 고려해 COCO JSON에도 바운딩박스 좌표를 남겨두었기 때문에 해당 작업에서도 관련 부분을 추가 생성함.

**#4. 새로운 카테고리 추가**
- "Person"과 "Vehicle"이라는 새로운 카테고리를 COCO 데이터에 추가함. 만약 기존 카테고리에 동일한 카테고리가 없다면 새로 추가하면 됨.

**#5. 수정된 JSON 파일 저장**
- 최종적으로 '_coco_updated.json 형식'으로 저장되며 하단에 'image_10_coco_updated.json' 일부분 첨부함. 
    - 첫 번째 좌표: 1161.00096, 570.00024 — 객체 경계의 시작점.
    - 두 번째 좌표: 1161.00096, 578.99988 — 첫 번째 점에서 아래로 이동.
    - 세 번째 좌표: 1154.99904, 585.00036 — 두 번째 점에서 왼쪽으로 이동.


<br>

**<image_10_coco_updated.json - categories ID 4(Vehicles)>**
```python
 ...
 {
    "id": 5,
    "image_id": 0,
    "category_id": 4,
    "segmentation": [
        [
            1161.00096,
            570.00024,
            1161.00096,
            578.99988,
            1154.99904,
            585.00036,
            1148.99904,
            585.00036,
            1146.0,
            587.99952
        ]
    ],
    "area": 67436.87579988486,
    "bbox": [
        1013.9999999999999,
        570.00024,
        380.99904000000026,
        177.00012000000004
    ],
    "iscrowd": 0
}

...


------------------------------------------------------------------------------------------------------------------------------------------------------

### 4.  **`one_coco_yolo.py`로 카테고리 4개 고정**

기존에는 `onecoco_2.py` 파일처럼 COCO JSON 파일에서 카테고리를 추가할 때, 단순히 category['id'] 값이 category_id_mapping에 없는 경우 새로운 ID를 할당하는 방식을 사용했었음. 하지만 이 과정에서 다른 파일에서 동일한 이름의 카테고리가 다른 ID로 인식 후 매핑을 진행하며 카테고리가 4개가 아닌 38개로 증가하는 이슈가 발생함. (-> 모델 학습시 에러 발생함.)

In [ ]:
import json
import os

def merge_coco_jsons(input_dir, output_file, fixed_categories):
    # 고정된 카테고리 목록에서 카테고리 이름을 ID로 매핑
    category_map = {category['name']: category['id'] for category in fixed_categories}
    
    merged_coco = {
        "images": [],
        "annotations": [],
        "categories": fixed_categories
    }

    annotation_id = 1
    image_id = 1

    #1. JSON 파일 읽어오기
    for filename in os.listdir(input_dir):
        if filename.endswith("_coco.json") or filename.endswith("_updated.json"):
            json_path = os.path.join(input_dir, filename)
            with open(json_path, 'r', encoding='utf-8') as f:
                coco_data = json.load(f)
                
                #2. 원래 카테고리와 이름 매핑
                original_category_map = {category['id']: category['name'] for category in coco_data['categories']}
                
                #3. 이미지 데이터 병합
                if "images" in coco_data:
                    for image in coco_data["images"]:
                        new_image = image.copy()
                        new_image['id'] = image_id
                        merged_coco['images'].append(new_image)
                        
                        #4. 어노테이션 데이터 병합 및 카테고리 ID 매핑
                        if "annotations" in coco_data:
                            for annotation in coco_data["annotations"]:
                                if annotation['image_id'] == image['id']:
                                    new_annotation = annotation.copy()
                                    new_annotation['id'] = annotation_id
                                    new_annotation['image_id'] = image_id
                                    
                                    # 원래 카테고리 ID를 사용하여 이름을 찾고, 고정된 카테고리 ID로 매핑
                                    category_name = original_category_map.get(annotation['category_id'])
                                    if category_name and category_name in category_map:
                                        new_annotation['category_id'] = category_map[category_name]
                                    else:
                                        continue
                                    
                                    merged_coco['annotations'].append(new_annotation)
                                    annotation_id += 1
                        
                        image_id += 1
                        
    # JSON 구조 출력 테스트
    print(f"총 이미지 수: {len(merged_coco['images'])}")
    print(f"총 어노테이션 수: {len(merged_coco['annotations'])}")
    print(f"총 카테고리 수: {len(merged_coco['categories'])}")

    # 병합된 JSON 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(merged_coco, f, indent=4)

# 고정된 카테고리 목록
fixed_categories = [
    {"id": 1, "name": "Parking Space"},
    {"id": 2, "name": "Driveable Space"},
    {"id": 3, "name": "Person"},
    {"id": 4, "name": "Vehicle"}
]

# 사용 예시 
input_dir = '/home/elicer/runs/segment/predict/cocojson'
output_file = '/home/elicer/merged_data/merged_yolo_coco.json'
                

merge_coco_jsons(input_dir, output_file, fixed_categories)

**category_map = {category['name']: category['id'] for category in fixed_categories}**
- 따라서 'fixed_categories = [
    {"id": 1, "name": "Parking Space"},
    {"id": 2, "name": "Driveable Space"},
    {"id": 3, "name": "Person"},
    {"id": 4, "name": "Vehicle"}
]
'로 모든 파일에서 동일한 이름의 카테고리가 동일한 ID를 갖도록 강제하여 카테고리 수를 정확하게 고정시키는 작업을 진행함

<br>

**#1. COCO JSON 파일과 YOLO .txt 파일 매칭 및 JSON 데이터 로드**
- input_dir에서 '_coco.json' 또는 '_updated.json'으로 끝나는 파일들을 모두 읽어온 후 각 파일의 JSON 데이터를 'coco_data' 변수로 로드함.

**#2. 원래 카테고리와 이름 매핑**
- 현재 읽고 있는 JSON 파일에서 정의된 카테고리 ID와 이름을 매핑하는 딕셔너리(original_category_map)를 생성함. (카테고리 ID 일관성 유지!)
- 예를 들어, "Vehicle"이라는 카테고리가 ID 10으로 정의되어 있다면, 이 딕셔너리는 {10: "Vehicle"}와 같은 형태를 가짐.

**#3. 이미지 데이터 병합**
- 각 이미지를 merged_coco의 images 리스트에 추가함.

**#4. 어노테이션 데이터 병합 및 카테고리 ID 매핑**
- 먼저 각 어노테이션을 순회하면서, 현재 이미지에 속한 어노테이션을 merged_coco의 annotations 리스트에 추가함. 동시에 어노테이션도 역시 고유한 ID를 가지도록 새로운 ID(annotation_id)를 부여해줌. 
- original_category_map.get()으로 현재 카테고리 ID에 해당하는 이름(category_name)을 찾아낸 후 그 이름을 category_map에서 고정된 ID로 매핑함.(-> category_map은 고정된 카테고리 목록(fixed_categories)을 기반으로 {name: id} 형태로 정의.)
- 예를 들어, "Vehicle"이라는 이름이 category_map에서 ID 4로 고정되어 있다면, 해당 어노테이션의 카테고리 ID를 4로 설정하는 방식임.


------------------------------------------------------------------------------------------------------------------------------------------------------

### 5. **`check.py` 파일 실행(선택사항)** 

In [ ]:
from pycocotools.coco import COCO

# annotation 파일 경로를 입력하세요.
coco_annotation_file = '/home/elicer/merged_data/merged_yolo_coco.json'

# COCO API 초기화
coco = COCO(coco_annotation_file)

# 모든 카테고리 확인
categories = coco.loadCats(coco.getCatIds())
for category in categories:
    print(f"Category ID: {category['id']}, Name: {category['name']}")

- 카테고리 4개 정상 출력 확인함. 
- 이후 `modeltrain_yolo.py`로 학습 진행.

```python
Category ID: 1, Name: Parking Space
Category ID: 2, Name: Driveable Space
Category ID: 3, Name: Person
Category ID: 4, Name: Vehicle
